# Age prediction using street view images
Link: https://www.kaggle.com/code/gcdatkin/age-prediction-from-images-cnn-regression/notebook 

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [2]:
image_dir = Path('/workspace/workspace/ufo-prediction/image_data_NLD')

In [3]:
filepaths = pd.Series(list(image_dir.glob('**/*.jpg')), name='Filepath').astype(str)

def extract_age_and_name(filepath):
    filename = os.path.basename(filepath)  # Get the filename from the filepath
    age_and_name = filename.split('.')[0]  # Split by dot and take the first part
    age = ''.join(filter(str.isdigit, age_and_name))[:4]  # Extract first 4 digits for age
    if age:  # Ensure age string is not empty
        age = int(age)
    else:  # Default age if no digits found
        age = 0
    return age

ages = pd.Series(filepaths.apply(lambda x: extract_age_and_name(x)), name='Age')

# Filter to include only ages above 1900
filtered_ages = ages[ages > 1900]

# Ensure we only work with filepaths that have a corresponding age above 1900
filtered_filepaths = filepaths[ages > 1900]

images = pd.concat([filtered_filepaths, filtered_ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
images

,Filepath,Age
0,/workspace/workspace/ufo-prediction/image_data...,1986
1,/workspace/workspace/ufo-prediction/image_data...,1966
2,/workspace/workspace/ufo-prediction/image_data...,1956
3,/workspace/workspace/ufo-prediction/image_data...,1978
4,/workspace/workspace/ufo-prediction/image_data...,1948
...,...,...
7056,/workspace/workspace/ufo-prediction/image_data...,1932
7057,/workspace/workspace/ufo-prediction/image_data...,1975
7058,/workspace/workspace/ufo-prediction/image_data...,1978
7059,/workspace/workspace/ufo-prediction/image_data...,2010


In [5]:
# Split into train and test set
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [7]:
train_df

,Filepath,Age
1351,/workspace/workspace/ufo-prediction/image_data...,1989
2048,/workspace/workspace/ufo-prediction/image_data...,1915
6235,/workspace/workspace/ufo-prediction/image_data...,1958
3407,/workspace/workspace/ufo-prediction/image_data...,1986
2441,/workspace/workspace/ufo-prediction/image_data...,1973
...,...,...
905,/workspace/workspace/ufo-prediction/image_data...,1989
5192,/workspace/workspace/ufo-prediction/image_data...,1934
3980,/workspace/workspace/ufo-prediction/image_data...,1949
235,/workspace/workspace/ufo-prediction/image_data...,1958


In [8]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 3954 validated image filenames.
Found 988 validated image filenames.
Found 2119 validated image filenames.


# Training

In [9]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )
    ]
)

Epoch 1/1000
124/124 [==============================] - 9s 30ms/step - loss: 1929320.0000 - val_loss: 96782.8438
Epoch 2/1000
124/124 [==============================] - 3s 20ms/step - loss: 88799.5625 - val_loss: 89264.3281
Epoch 3/1000
124/124 [==============================] - 3s 21ms/step - loss: 79585.6641 - val_loss: 77071.1719
Epoch 4/1000
124/124 [==============================] - 3s 20ms/step - loss: 66335.0234 - val_loss: 61963.2031
Epoch 5/1000
124/124 [==============================] - 3s 21ms/step - loss: 51502.9844 - val_loss: 46907.2656
Epoch 6/1000
124/124 [==============================] - 3s 20ms/step - loss: 36109.0625 - val_loss: 29591.9551
Epoch 7/1000
124/124 [==============================] - 3s 20ms/step - loss: 21310.1191 - val_loss: 15918.7002
Epoch 8/1000
124/124 [==============================] - 3s 20ms/step - loss: 10166.6543 - val_loss: 6992.9829
Epoch 9/1000
124/124 [==============================] - 2s 20ms/step - loss: 4891.4023 - val_loss: 3933.3938
Ep

# Results

In [10]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

67/67 [==============================] - 2s 21ms/step
     Test RMSE: 30.92044
Test R^2 Score: -0.02141


In [11]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 30.59473
